Load libraries and create df

In [1]:
import pandas as pd
from IPython.display import display
import os
import shutil
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py')
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)

Filter the current Week then rename the TTO and TTR column. Then insert new columns needed for the later part

Make sure that nnewdf = sladf[sladf['Week']==24] <-- current work week is updated

In [2]:
nnewdf = sladf[sladf['Week']==26]
nnewdf = nnewdf.rename( columns = {'TTR Result\n(Excluding Pending Time)': 'TTR'}, inplace = False)
nnewdf = nnewdf.rename( columns = {'TTO Result (Overall)': 'TTO'}, inplace = False )

nnewdf.insert(48, "MetricTTR","TTR Overall", False) #Helper Column
nnewdf.insert(49, "MetricTTO","TTO Overall", False) #Helper Column


Create a pivot for TTR and TTO

In [3]:
TTRpivot = pd.pivot_table(nnewdf, index=['MetricTTR'], columns=['TTR',], values=['Ticket No'], aggfunc='count')
TTOpivot = pd.pivot_table(nnewdf, index=['MetricTTO'], columns=['TTO',], values=['Ticket No'], aggfunc='count')


Remove the unwanted column level of the pivot table

In [4]:
TTRpivot.columns = TTRpivot.columns.droplevel(0)
TTRpivot.columns.name = None
TTRdf = TTRpivot.reset_index()

TTOpivot.columns = TTOpivot.columns.droplevel(0)
TTOpivot.columns.name = None
TTOdf = TTOpivot.reset_index()


Compute the percentage of TTR and TTO

In [5]:
TTRBeyond = TTRdf["Beyond TAT"].astype(int)
TTRWithin = TTRdf["Within TAT"].astype(int)
TTRsum = TTRBeyond + TTRWithin
TTRdf["Beyond TAT Percent"] = TTRBeyond / TTRsum
TTRdf["Within TAT Percent"] = TTRWithin / TTRsum

TTOBeyond = TTOdf["Beyond TAT"].astype(int)
TTOWithin = TTOdf["Within TAT"].astype(int)
TTOsum = TTOBeyond + TTOWithin
TTOdf["Beyond TAT Percent"] = TTOBeyond / TTRsum
TTOdf["Within TAT Percent"] = TTOWithin / TTRsum

Merge the 2 separate table into 1

In [6]:
TTRdf = TTRdf.rename( columns = {'MetricTTR': 'Metric'})

TTOdf = TTOdf.rename( columns = {'MetricTTO': 'Metric'})

frames = [TTRdf, TTOdf]
  
SLA_perf = pd.concat(frames)
display(SLA_perf)


,Metric,Beyond TAT,Within TAT,Beyond TAT Percent,Within TAT Percent
0,TTR Overall,456,1196,0.276029,0.723971
0,TTO Overall,250,1401,0.151332,0.848063


Save the df into an excel file 

In [9]:
FilePath = r'C:\repo\SLA_Perf.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FilePath, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
SLA_perf.to_excel(writer, sheet_name='SLA', index = False)
writer.save()
